In [1]:
from wallet_database_sqlite3 import Sqlite3Wallet

In [2]:
db = Sqlite3Wallet()

connection with database made.


In [3]:
db.conn

In [4]:
#db.erase_database()

database is locked
database is locked


True

In [6]:
res = db.create_tables()
res

True

In [4]:
res = db.does_table_exist("Wallets")
res

[('Wallets',)]

In [9]:
#f = db.cursor().execute("DROP TABLE Addresses")
#db.create_contact_table()
contact_list_raw = db.get_all_contacts()
contact_list_raw
contact_list  = [x[1]+" "+x[0] for x in contact_list_raw if x[1].startswith("o") or x[0].startswith("")]
contact_list

SELECT first_name, last_name, phone, position, xpub FROM Contacts



['perez pepito']

In [8]:
#db.close()

In [3]:
xprv ="tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP"
words = 'b"engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"'[1:]
res = db.new_wallet(xprv, words, "from jupyter notebook")

INSERT INTO Wallets (xprv, words, name)
 VALUES('tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP', "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning", 'from jupyter notebook') ;


In [4]:
res

True

In [5]:
res = db.new_address("first_address", "m/0H/0H/", 0, 0, xprv)
res

True

In [12]:
res = db.new_utxo("first_address", 22222, "transaction_0", 0, confirmed=6)
res

True

In [13]:
utxos = db.look_for_coins(xprv)

In [14]:
utxos

[('transaction_0', 0, 22222)]

In [15]:
unused_addresses = db.get_unused_addresses(xprv)
unused_addresses

[]

In [6]:
res = db.new_address("second_address", "m/0H/0H/", 1, 0, xprv)
res

True

In [11]:
res = db.new_address("Thirteenth_address", "m/0H/0H/", 13, 0, xprv)
res

True

In [17]:
unused_addresses = db.get_unused_addresses(xprv, 2,2)
unused_addresses

SELECT address 
FROM Addresses WHERE
NOT EXISTS(
SELECT 1 
 FROM Utxos
WHERE Utxos.address = Addresses.address) 
AND
 Addresses.wallet = 'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP'
AND
 (1591753283 - Addresses.created) > 0 AND (1591753283 - Addresses.created) < 172800 ;


[('second_address',)]

In [18]:
db.get_all_addresses(xprv)

[('first_address',), ('second_address',)]

In [18]:
f = db.conn.cursor().execute("SELECT * FROM Addresses")

In [19]:
f.fetchall()

[('first_address',
  0,
  "m/0'/0'/",
  0,
  1591752001,
  'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP'),
 ('second_address',
  1,
  "m/0'/0'/",
  1,
  1591752006,
  'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP')]

In [20]:
db.new_tx("tx_n1",[("transaction_0",0)],[(1000,"script_public_key of my first transaction")])

True

In [21]:
g = db.conn.cursor().execute("SELECT * FROM Transactions")
g.fetchall()

[('tx_n1', 0, 1, 0, 1591752025)]

In [22]:
h = db.conn.cursor().execute("SELECT * FROM Tx_Ins")
h.fetchall()

[('transaction_0', 0, 'tx_n1')]

In [3]:
i = db.conn.cursor().execute("SELECT * FROM Tx_Outs")
i.fetchall()

[(0,
  1000,
  '00508e440dc876e769e0dd3e2f61f8e7d33ce90a2e8f023c9aa9f0b8e48c4888',
  'This is a test ScriptPubKey')]

In [24]:
db.conn.cursor().execute("DROP TABLE Wallets")

In [25]:
db.does_table_exist("Addresses")

[('Addresses',)]

In [4]:
db.close_database()

In [28]:
account = "m/0H/1H/"
g = db.conn.cursor().execute(f"SELECT MAX(acc_index) FROM Addresses WHERE wallet='{xprv}' AND path='{account}';")
j = g.fetchall()[0][0]
j

In [29]:
if j is None:
    print("yes")

yes


In [8]:
wallet = "tprv8ZgxMBicQKsPcv5EhtDXAeg624gwoQpnB55T1qaUPbgzjUrziZ9VkM2es6MsMBe7tRpvnoGWb9RFoKjhpkovSepJiMFX5NWBVtcpWcd96pp"
query1 = f"SELECT tx_id, out_index, amount\n FROM Utxos INNER JOIN Addresses \n"
#query1 = f"SELECT tx_id, out_index, amount\n FROM Utxos;"
query2 = f"ON Utxos.address = Addresses.address\nWHERE Utxos.spent = 0 AND Addresses.wallet = '{wallet}';"
#query2 = f"ON Utxos.address = Addresses.address\nWHERE Utxos.spent = 1;"
query = query1 + query2
i = db.conn.cursor().execute(query)
i.fetchall()

[('ce8989c24a699924ecf8281b0b88f1f04de9a98ada0d24fd24f053a2ece21b5e',
  1,
  1000000),
 ('71ae64055a84d489aa9f92bb53ad1d76dd846cbafc614a16893c1adf5023c6eb',
  1,
  22000)]

In [13]:
query= "UPDATE Utxos SET spent = 1 WHERE tx_id = 'fb879418a9f754cc4d050c37fcb2af4e0b8e65a7abc7213828f0d507cd1c24e2' AND out_index = 1"
i = db.conn.cursor().execute(query)

In [14]:
query= "SELECT spent FROM Utxos WHERE tx_id = 'fb879418a9f754cc4d050c37fcb2af4e0b8e65a7abc7213828f0d507cd1c24e2' AND out_index = 1"
i = db.conn.cursor().execute(query)
i.fetchall()

[(1,)]

In [15]:
db.conn.commit()


In [10]:
from wallet_database_sqlite3 import Sqlite3Environment


env = Sqlite3Environment()
env

connection with database made.


In [11]:
env.new_key("BLOCKCYPHER_API_KEY","d3e470c810b6440ca60e57f1f60939c6")
env.new_key("CC_API","e57bbc8169583b0ebfba6e517836f199eab4ffa74a752d54fcfd2cd0cc385641")

INSERT INTO Keys (name, key)
VALUES('BLOCKCYPHER_API_KEY', 'd3e470c810b6440ca60e57f1f60939c6') ;
INSERT INTO Keys (name, key)
VALUES('CC_API', 'e57bbc8169583b0ebfba6e517836f199eab4ffa74a752d54fcfd2cd0cc385641') ;


True

In [12]:
i = env.conn.cursor().execute("SELECT * FROM Keys")
i.fetchall()

[('BLOCKCYPHER_API_KEY', 'd3e470c810b6440ca60e57f1f60939c6'),
 ('CC_API',
  'e57bbc8169583b0ebfba6e517836f199eab4ffa74a752d54fcfd2cd0cc385641')]

In [19]:
query= "SELECT * FROM SHDSafeWallet"
i = db.conn.cursor().execute(query)
i.fetchall()

[('',
  '[242181816824935352779949905339585673702570459364503992825384375679130606102141, 243939266812165980563239760114596239186104406092581261489726331066308569928747, 290372254706026096153481489592931453085780088683353495329679236536122868177482, 376644786442849392304182314585976636972636060283971456632342867625859504056500, 461222389963770672910023206337944803706847222730930779040024025202330161562314]',
  2,
  6,
  'None',
  'tprv8ghfWd73PvMPqRazzw1m6Vxq6TUzVmbuiyfQ4frvAXgM8kPidJ7QWDWPRmwF9J3da2EhRnLJjiwwFKYAo4bW8p82JRntd8MtKydwTpDoiGj',
  'tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf',
  'p2wsh',
  1,
  1,
  'None',
  -1),
 ('200823',
  '[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 34904519403285236524485843581086005758991025139939414745136436266219764179988]',
  2,
  4,
  'None',
  'tprv8oQnacENnx7r

In [13]:
query= "SELECT tx_id,out_index FROM Tx_Ins WHERE spent_by = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a'"
i = db.conn.cursor().execute(query)
i.fetchall()

[('5324623d891ff225b41570d09320770c30d9291a0193cda09504a115fb661d1c', 0)]

In [15]:
query1 = "SELECT spent FROM Utxos WHERE tx_id = ("
query2 = "SELECT tx_id FROM Tx_Ins WHERE spent_by = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a') "
query3 = "AND out_index = (SELECT out_index FROM Tx_Ins WHERE spent_by = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a')"

query = query1 + query2 + query3
i = db.conn.cursor().execute(query)
i.fetchall()

[(1,)]

In [ ]:
query1 = "UPDATE Utxos SET spent = 0 WHERE ("
query2 = "SELECT tx_id,out_index FROM Tx_Ins WHERE spent_by = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a')"
query = query1 + query2
i = db.conn.cursor().execute(query)
i.fetchall()

In [14]:
query = "ALTER TABLE Contacts ADD COLUMN safe_pubkey text"
w = db.conn.cursor().execute(query)

In [15]:
db.conn.commit()


In [3]:
query = "SELECT * \nFROM addresses"
w = db.conn.cursor().execute(query)
w.description

(('address', None, None, None, None, None, None),
 ('acc_index', None, None, None, None, None, None),
 ('path', None, None, None, None, None, None),
 ('change_addr', None, None, None, None, None, None),
 ('created', None, None, None, None, None, None),
 ('wallet', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None),
 ('safe_index', None, None, None, None, None, None))

In [20]:
query = "ALTER TABLE SHDSafeWallet ADD COLUMN level1pubkeys text"
w = db.conn.cursor().execute(query)
db.conn.commit()

In [11]:
query = "SELECT * FROM wallets WHERE 1=0"
w = db.conn.cursor().execute(query)
w.description

(('xprv', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('words', None, None, None, None, None, None),
 ('child', None, None, None, None, None, None))

In [3]:
w = db.create_SHDSafeWallet_table()

In [4]:
db.conn.commit()

In [4]:
query = "SELECT * FROM SHDSafeWallet"
w = db.conn.cursor().execute(query)
w.fetchall()

[('test1_master_wallet',
  '[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]',
  2,
  4,
  'None',
  'tprv8fVXQGqBdhmo7Lymx7JARMiyt59rg9JuuS5uxVsgqKCpQt5wSD9G6HY8uVQs5CPNV5i6qt1vWes78DmmbrKa1gncZXpaL3m71GT3RsRcWda',
  'tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv',
  'p2wsh',
  1,
  1,
  'None',
  -1)]

In [4]:
query = "SELECT * FROM Transactions WHERE tx_id = '7b11ce3edff6f0d7426c5bc76538854faa80a53d1039b5cf1c379420abdbdf49' "
w = db.conn.cursor().execute(query)
w.fetchall()

[('7b11ce3edff6f0d7426c5bc76538854faa80a53d1039b5cf1c379420abdbdf49',
  0,
  1,
  0,
  1598472061)]

In [5]:
w.description

(('tx_id', None, None, None, None, None, None),
 ('lock_time', None, None, None, None, None, None),
 ('version', None, None, None, None, None, None),
 ('n_confirmations', None, None, None, None, None, None),
 ('created', None, None, None, None, None, None))

In [4]:
w = db.create_HDMWallet_table()
db.conn.commit()

In [7]:
query = "SELECT * FROM Tx_Outs"
w = db.conn.cursor().execute(query)
w.fetchall()

[(0,
  1000,
  '00508e440dc876e769e0dd3e2f61f8e7d33ce90a2e8f023c9aa9f0b8e48c4888',
  'This is a test ScriptPubKey'),
 (0,
  21300,
  '3263618890ceb9f3b7a18b493b7821707e55a1acde2700aa3e917fa3af2ea165',
  'OP_0 ef6232e84397e91a7e22b17cb97f5cec89d088bd'),
 (1,
  73786,
  '3263618890ceb9f3b7a18b493b7821707e55a1acde2700aa3e917fa3af2ea165',
  'OP_0 2368d4f87d9ab5510116bf1b1c6ee8cfe4d1c8b3'),
 (0,
  2345,
  '4896821137baece4df4b858ff244ebe3b7aad68663f9ffd1223ebcf385b77a45',
  'OP_0 73e624baac78eba72c25a0909318f988506295e581f8f6236a32d02fbdb08fbe'),
 (1,
  986323,
  '4896821137baece4df4b858ff244ebe3b7aad68663f9ffd1223ebcf385b77a45',
  'OP_0 e1bee14017102bcbd87ea64e0e3fb1716c38d831'),
 (0,
  33300,
  '4cc1d0bf1c1d7276609e9539b6208856c468019efc8f3ccd57b43517bc12917a',
  'OP_0 ef6232e84397e91a7e22b17cb97f5cec89d088bd'),
 (1,
  174,
  '4cc1d0bf1c1d7276609e9539b6208856c468019efc8f3ccd57b43517bc12917a',
  'OP_0 847ba3d304bfaf3edb8d21f2b468782c7d0291e1'),
 (0,
  50000,
  '568e5a300ad56625fcd2480d81b6

In [4]:
db.create_partial_transaction_table()
db.create_partial_tx_in_table()
db.create_partial_tx_out_table()

True